# Build NN from Scratchm
https://www.udemy.com/course/build-neural-networks-from-scratch-with-python-step-by-step/

# Common Functions

In [ ]:
import numpy as np

In [ ]:
# def bin_classification(x):
#     return 1 if x >= 0.5 else 0
# bin_class_vectorised = np.vectorize(bin_classification)


# def regression_predict(w, x):
#     return w * x
# regression_predict_vectorized = np.vectorize(regression_predict)    

In [12]:
import numpy as np

class Sigmoid():
    activation_type = 'Sigmoid'
    
    def execute(self, input_matrix):
        z = np.exp(-input_matrix)
        sig = 1 / (1 + z)      
        return sig

class Sigmoid_Stable():
    activation_type = 'Sigmoid_Stable'
    
    def execute(self, input_matrix):
        sig = np.where(input_matrix < 0, np.exp(input_matrix)/(1 + np.exp(input_matrix)), 
                       1/(1 + np.exp(-input_matrix)))
        return sig
    
    

class Model():
    def __init__(self, layers):
        self._layers = layers  # a list of layer objects
        
    def get_model_details(self):
        model_details = []
        for layer in self._layers:
            tDict = {}
            tDict['name'] = layer.get_layer_name()
            model_details.append(tDict)
        return model_details
    
    def matrix_mul_dims_ok(layer_prev, weights):
        # class method
        m, n1 = layer_prev.shape
        print(f'\n\nLprev.shape: \tm = {m} \tby n = {n1}')

        n2, p = weights.shape
        print(f'x.shape: \tn = {n2} \tby p = {p}')

        if n1 != n2:
            print('n1 != n2, matrix multiplication will fail.\n')
            print('either A or B should be reshaped if possible.')
            return False
        else:
            return True
        
    #     def forward(self, X):
#         self._matrix = np.sum(np.dot(X, self._weights_matrix), axis=0)


class Layer():
    layer_type = 'Base Layer'
    
    def __init__(self, layer_name, nodes_prev_layer, nodes_in_layer):
        self._layer_name = layer_name
        self._nodes_prev_layer = nodes_prev_layer
        self._nodes = nodes_in_layer
    
    def get_layer_details(self):
        layer_details = {}
        layer_details['name'] = self._layer_name
        layer_details['layer_type'] = self.layer_type  # uses child class's class variable
        layer_details['num_nodes_prev'] = self._nodes_prev_layer
        layer_details['num_nodes'] = self._nodes
        layer_details['layer_shape'] = self._layer_matrix.shape
        layer_details['layer_matrix'] = self._layer_matrix
        return layer_details
    
    def print_layer_details(self):
        layer_details = self.get_layer_details()
        print()
        print('*'*50)
        print(f"name: \t\t{layer_details['name']}")
        print(f"layer_type: \t{layer_details['layer_type']}")
        print(f"num_nodes_prev: {layer_details['num_nodes_prev']}")
        print(f"num_nodes: \t{layer_details['num_nodes']}")
        print(f"layer_shape: \t{layer_details['layer_shape']}")
        
    def get_layer_matrix(self):
        return self._layer_matrix

    
class Input_Layer(Layer):
    layer_type = 'Input Layer'
    
    def __init__(self, layer_name, nodes_prev_layer, nodes_in_layer, data):
        super().__init__(layer_name, nodes_prev_layer, nodes_in_layer)  
        self._data = data
        self._layer_matrix = np.array(data).reshape(-1, self._nodes)  # representation of this layer

    def data_wrangling_fns(self):
        # placeholder - otherwise, no reason to have a child class just for input
        pass
    
        
class Fully_Connected_Layer(Layer):
    layer_type = 'Fully Connected Layer'
    
    def __init__(self, layer_name, nodes_prev_layer, nodes_in_layer, act_fn):
        super().__init__(layer_name, nodes_prev_layer, nodes_in_layer)  
        
        self._weights_matrix = np.random.rand(self._nodes_prev_layer * self._nodes)
        self._weights_matrix = self._weights_matrix.reshape(self._nodes_prev_layer, self._nodes)
               
        self._layer_matrix = np.zeros(self._nodes).reshape(1,self._nodes)  # representation of this layer
        
        self.act_fn = act_fn

    def get_layer_details(self):
        layer_details = super().get_layer_details()
        layer_details['weights_shape'] = self._weights_matrix.shape
        layer_details['weights_matrix'] = self._weights_matrix
        layer_details['activation'] = self.act_fn.activation_type
        return layer_details
    
    def print_layer_details(self):
        super().print_layer_details()
        layer_details = self.get_layer_details()
        print(f"weights_shape: \t{layer_details['weights_shape']}")
        print(f"activation: \t{layer_details['activation']}")

    def get_weights_matrix(self):
        return self._weights_matrix
    
    def forward(self, X):
        if X.ndim == 1:
            X = X.reshape(1,-1)
            
        #self._layer_matrix = np.sum(np.dot(X, self._weights_matrix), axis=0)        
        self._layer_matrix = np.sum(np.dot(X, self._weights_matrix), axis=0)        
        
        # print(f'before act fn: {self.get_layer_matrix()}')        
        self._layer_matrix = self.act_fn.execute(self._layer_matrix)                
        
        if self._layer_matrix.ndim ==1:
            self._layer_matrix = self._layer_matrix.reshape(1,-1)            
    
    def test(self):
        self.act_fn.execute(888)

        
class Output_Binary_Classification(Fully_Connected_Layer):
    layer_type = 'Output_Binary_Classification'
    
    def __init__(self, layer_name, nodes_prev_layer, nodes_in_layer, act_fn):
        nodes_in_layer = 1 # hardcode coz binary classification
        
        super().__init__(layer_name, nodes_prev_layer, nodes_in_layer, act_fn)  
        
        self._predicted_class = None
    
    def get_probability_matrix(self):
        return self._layer_matrix
        
    def predict(self, threshold=0.5):
        # returns an array of array, hence [0][0] to get the int
        print(f"threshold: {threshold}")
        self._predicted_class = np.where(self._layer_matrix >= threshold, 1, 0)[0][0]
        return self._predicted_class

# not implemented, act fn sh be softmax    
# class Output_MultiClass_Classification(Fully_Connected_Layer): # predict 1 class out of multiple classes 
# class Output_MultiLabel_Classification(Fully_Connected_Layer): # predict p-values of 1 or more classes
                                                                 #   an input can belong to >1 class

class Output_Regression(Fully_Connected_Layer):
    layer_type = 'Output_Regression'
    
    def __init__(self, layer_name, nodes_prev_layer, nodes_in_layer, act_fn):
        nodes_in_layer = 1 # hardcode coz regression
        
        super().__init__(layer_name, nodes_prev_layer, nodes_in_layer, act_fn)  
        
        
        
    def predict(self):
        # returns an array of array, hence [0][0] to get the int
        return self._layer_matrix[0][0]
    
    
data = [
    [1,1,1],
    [2,2,2],
    [3,3,3],
    [4,4,4]]
                 
In = Input_Layer('Input', 0, 3, data)
In.print_layer_details()
print(In.get_layer_matrix())

H1 = Fully_Connected_Layer('H1', 3, 5, Sigmoid())
H1.print_layer_details()
H1.forward(In.get_layer_matrix())
print(f"after forward pass:\n{H1.get_layer_matrix()}")

H2 = Fully_Connected_Layer('H2', 5, 8, Sigmoid())
H2.print_layer_details()
H2.forward(H1.get_layer_matrix())
print(f"after forward pass:\n{H2.get_layer_matrix()}")

#output_type = "Binary Classification"
output_type = 'Regression'

if output_type == "Binary Classification":
    Out = Output_Binary_Classification('Output', 8, 1, Sigmoid())
    Out.print_layer_details()
    Out.forward(H2.get_layer_matrix())
    print(f"after forward pass:\n{Out.get_layer_matrix()}")
    pred = Out.predict(threshold=0.5)
    print(f"The model predicts: 'class {pred}'.")
elif output_type == 'Regression':
    Out = Output_Regression('Output', 8, 1, Sigmoid())
    Out.print_layer_details()
    Out.forward(H2.get_layer_matrix())
    print(f"after forward pass:\n{Out.get_layer_matrix()}")
    pred = Out.predict()
    print(f"The model predicts: '{pred:0.2f}")




**************************************************
name: 		Input
layer_type: 	Input Layer
num_nodes_prev: 0
num_nodes: 	3
layer_shape: 	(4, 3)
[[1 1 1]
 [2 2 2]
 [3 3 3]
 [4 4 4]]

**************************************************
name: 		H1
layer_type: 	Fully Connected Layer
num_nodes_prev: 3
num_nodes: 	5
layer_shape: 	(1, 5)
weights_shape: 	(3, 5)
activation: 	Sigmoid
after forward pass:
[[0.99998313 0.99999999 0.99999834 1.         1.        ]]

**************************************************
name: 		H2
layer_type: 	Fully Connected Layer
num_nodes_prev: 5
num_nodes: 	8
layer_shape: 	(1, 8)
weights_shape: 	(5, 8)
activation: 	Sigmoid
after forward pass:
[[0.96280489 0.92340291 0.93246346 0.91241051 0.92581299 0.94308898
  0.92944318 0.91621091]]

**************************************************
name: 		Output
layer_type: 	Output_Regression
num_nodes_prev: 8
num_nodes: 	1
layer_shape: 	(1, 1)
weights_shape: 	(8, 1)
activation: 	Sigmoid
after forward pass:
[[0.97736036]]
The mo